<a href="https://colab.research.google.com/github/bhuvnk/EVA/blob/master/Assignment14/Paperspace_RunAssignment14_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bhuvnesh Kumar  // Saturday 11am batch 

Ran on Paperspace 
* V100  
* 30gb ram
* 8vcpu

* Acc: 94.04
* time: 292 Sec
* extra augmentation: cutout


In [0]:
import numpy as np
import time, math

import tensorflow as tf
import tensorflow.contrib.eager as tfe

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
!nvidia-smi

Sat Aug 24 17:45:44 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    36W / 300W |     80MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import multiprocessing

multiprocessing.cpu_count()

8

In [0]:
tf.enable_eager_execution()

In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float16, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.2):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float16))
    return loss, correct

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)

num_classes = 10
# y_train = tf.keras.utils.to_categorical(y_train, num_classes, dtype='float16')
# y_test = tf.keras.utils.to_categorical(y_test, num_classes, dtype='float16')
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float16') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

# Cutout

In [0]:
def replace_slice(input_: tf.Tensor, replacement, begin) -> tf.Tensor:
    inp_shape = tf.shape(input_)
    size = tf.shape(replacement)
    padding = tf.stack([begin, inp_shape - (begin + size)], axis=1)
    replacement_pad = tf.pad(replacement, padding)
#     replacement_pad = tf.cast(replacement_pad, dtype=tf.float16)
    mask = tf.pad(tf.ones_like(replacement, dtype=tf.bool), padding)
    return tf.where(mask, replacement_pad, input_)

def cutout(x: tf.Tensor, h: int=8, w: int=8, c: int = 3) -> tf.Tensor:
    shape = tf.shape(x)
    x0 = tf.random.uniform([], 0, shape[0] + 1 - h, dtype=tf.int32)
    y0 = tf.random.uniform([], 0, shape[1] + 1 - w, dtype=tf.int32)
    
    x = replace_slice(x, tf.zeros([h, w, c], dtype = tf.float16), [x0, y0, 0])
#     x = replace_slice(x, tf.constant(1,shape = [h,w,c],dtype = tf.float16)*train_mean, [x0, y0, 0])
    return x

# Training the model

# With
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, 21, EPOCHS], [0, LEARNING_RATE,0.021, 0])[0]


In [0]:
model = DavidNet()
#model.summary()
batches_per_epoch = len_train//BATCH_SIZE + 1
# LEARNING_RATE=0.4
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, 21, EPOCHS], [0, LEARNING_RATE,0.021, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
# opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (cutout(tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3]))), y)


t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

mom_state = {}

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug, num_parallel_calls=8).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
#   for (x, y) in tqdm(train_set):
  for (x, y) in train_set:
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    
    global_step.assign_add(1)
    lr = lr_func()

    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
      
      if v not in mom_state:
        state = tf.zeros_like(g)
      else:
        state = mom_state[v]
        
      state = state * MOMENTUM + g
      g += state * MOMENTUM
      v.assign_add(g * (-lr))
      mom_state[v] = state

#     opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

W0824 17:45:55.403690 140525221713728 deprecation.py:323] From <ipython-input-12-c9dc21f47e08>:8: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


epoch: 1 lr: 0.08 train loss: 1.538935 train acc: 0.44014 val loss: 1.2504 val acc: 0.5856 time: 17.21765923500061
epoch: 2 lr: 0.16 train loss: 0.99907 train acc: 0.64874 val loss: 1.056825 val acc: 0.6439 time: 29.396451234817505
epoch: 3 lr: 0.24 train loss: 0.793065 train acc: 0.72694 val loss: 1.076775 val acc: 0.6747 time: 41.439059257507324
epoch: 4 lr: 0.32 train loss: 0.659765 train acc: 0.77282 val loss: 0.900975 val acc: 0.7148 time: 53.11166858673096
epoch: 5 lr: 0.4 train loss: 0.576995 train acc: 0.79802 val loss: 0.7360625 val acc: 0.7521 time: 64.86671018600464
epoch: 6 lr: 0.37631250000000005 train loss: 0.500575 train acc: 0.82768 val loss: 0.55155 val acc: 0.8086 time: 76.6717689037323
epoch: 7 lr: 0.352625 train loss: 0.44165 train acc: 0.84704 val loss: 0.6141125 val acc: 0.7959 time: 88.26883912086487
epoch: 8 lr: 0.3289375 train loss: 0.4107225 train acc: 0.85878 val loss: 0.5123 val acc: 0.8252 time: 100.2941381931305
epoch: 9 lr: 0.30525 train loss: 0.37867 tra